# Memory

<div style="display: flex; justify-content: flex-start; gap: 10px;">
  <img src="./assets/LC_Memory_after.png" style="width:300px; border:1px solid #ccc; border-radius:6px;">
</div>

Persisting messages, or 'agent state' between invocations of the agent.

## Setup

Load and/or check for needed environmental variables

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

OPENAI_API_KEY=****eJgA
LANGSMITH_API_KEY=****2eed
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ject


In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [3]:
from dataclasses import dataclass

@dataclass
class RuntimeContext:
    db: SQLDatabase

In [4]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [5]:
SYSTEM_PROMPT = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

In [6]:
from langchain.agents import create_agent

from dotenv import load_dotenv
import os
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),  # or your deployment
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),  # or your api version
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

agent = create_agent(
    model=llm,
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
)

## Repeated Queries

In [7]:
question = "This is Frank Harris, What was the total on my last invoice?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
    context=RuntimeContext(db=db),
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

This is Frank Harris, What was the total on my last invoice?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_6QQfjj9gYsVRjqLWIYXQGCid)
 Call ID: call_6QQfjj9gYsVRjqLWIYXQGCid
  Args:
    query: SELECT total FROM invoices WHERE customer_name = 'Frank Harris' ORDER BY invoice_date DESC LIMIT 1;
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.OperationalError) no such table: invoices
[SQL: SELECT total FROM invoices WHERE customer_name = 'Frank Harris' ORDER BY invoice_date DESC LIMIT 1;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_25YtlSYVYXT1TqWumNWsnFM4)
 Call ID: call_25YtlSYVYXT1TqWumNWsnFM4
  Args:
    query: SELECT total_amount FROM billing WH

GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT

In [8]:
question = "What were the titles?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What were the titles?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_cmhGWD1P4Ta4WqeHaPGmq5ga)
 Call ID: call_cmhGWD1P4Ta4WqeHaPGmq5ga
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table' ORDER BY name LIMIT 5;
================================= Tool Message =================================
Name: execute_sql

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',)]
================================== Ai Message ==================================

Could you clarify which titles you mean—album titles, track titles, or something else? If you want album titles, I can list the first 5.


## Add memory

In [9]:
from langgraph.checkpoint.memory import InMemorySaver

In [10]:
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage

agent = create_agent(
    model="openai:gpt-5",
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
    context_schema=RuntimeContext,
    checkpointer=InMemorySaver(),
)

In [11]:
question = "This is Frank Harris, What was the total on my last invoice?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

This is Frank Harris, What was the total on my last invoice?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_6UxxtVjiPsxrp3dBYtn8eDXB)
 Call ID: call_6UxxtVjiPsxrp3dBYtn8eDXB
  Args:
    query: SELECT name, sql FROM sqlite_master WHERE type='table' AND (lower(name) LIKE '%invoice%' OR lower(name) LIKE '%customer%' OR lower(name) LIKE '%client%') ORDER BY name LIMIT 5;
================================= Tool Message =================================
Name: execute_sql

[('Customer', 'CREATE TABLE [Customer]\n(\n    [CustomerId] INTEGER  NOT NULL,\n    [FirstName] NVARCHAR(40)  NOT NULL,\n    [LastName] NVARCHAR(20)  NOT NULL,\n    [Company] NVARCHAR(80),\n    [Address] NVARCHAR(70),\n    [City] NVARCHAR(40),\n    [State] NVARCHAR(40),\n    [Country] NVARCHAR(40),\n    [PostalCode]...'), ('Invoice', 'CREATE TABLE [Invoice]\n(\n    [InvoiceId] IN

In [12]:
question = "What were the titles?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

What were the titles?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_NVIzXxF2vjAD1uAi2kI3CXl4)
 Call ID: call_NVIzXxF2vjAD1uAi2kI3CXl4
  Args:
    query: SELECT il.InvoiceLineId, t.Name AS Title
FROM InvoiceLine AS il
JOIN Track AS t ON t.TrackId = il.TrackId
WHERE il.InvoiceId = 374
ORDER BY il.InvoiceLineId
LIMIT 5;
================================= Tool Message =================================
Name: execute_sql

[(2021, 'Holier Than Thou'), (2022, 'Through The Never'), (2023, 'My Friend Of Misery'), (2024, 'The Wait'), (2025, 'Blitzkrieg')]
================================== Ai Message ==================================

The titles on your last invoice (ID 374) include:
- Holier Than Thou
- Through The Never
- My Friend Of Misery
- The Wait
- Blitzkrieg

Would you like the full list with quantities and prices?


## Try your own queries
Now that there is memory, check the agents recall!

In [ ]:
question = "Your Question Here?"
steps = []

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context=RuntimeContext(db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)